In [16]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os
os.chdir(os.path.dirname(os.getcwd()))
os.chdir('h:/programmer/2023/Oracle/')
print(os.path.dirname(os.getcwd()))
from datetime import datetime
import pandas as pd

h:\programmer\2023


In [17]:
# Факт на задание
fact = pd.read_csv("./Трек 1 - ПГК Оракул/fact_train_test.csv", sep=";", decimal=",", encoding="windows-1251")

fact["period"] = fact["period"].astype("datetime64[ns]")
# fact - уже обрезанная выборка

# Пример прогноза - наивный, копия последнего факта
result = []
last_known_fact_month = fact["period"].max()
test_periods = pd.date_range(start=last_known_fact_month, periods=6, freq='MS', inclusive='right')
for period in test_periods:
    print(period)
    month_forecast = fact[fact["period"] == last_known_fact_month].reset_index(drop=True)
    month_forecast["period"] = period
    result.append(month_forecast)
result = pd.concat(result).reset_index(drop=True)
result.rename(columns={"real_wagon_count": "forecast_wagon_count", "real_weight": "forecast_weight"}, inplace=True)
result.to_csv("./forecast_example.csv", index=False, sep=";", decimal=",", encoding="windows-1251")

2023-04-01 00:00:00
2023-05-01 00:00:00
2023-06-01 00:00:00
2023-07-01 00:00:00
2023-08-01 00:00:00


In [8]:
# Относительные ссылки, включая импорты, относительно корневой папки проекта
import os
os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd
import numpy as np

In [18]:
def add_master_data_mappings(df: pd.DataFrame) -> pd.DataFrame:
    # = Пути к справочникам - откорректировать если в реальной системе будут лежать по другому адресу =
    client_mapping_file = "./Трек 1 - ПГК Оракул/client_mapping.csv"
    freight_mapping_file = "./Трек 1 - ПГК Оракул/freight_mapping.csv"
    station_mapping_file = "./Трек 1 - ПГК Оракул/station_mapping.csv"

    # Клиент - холдинг
    client_mapping = pd.read_csv(
        client_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(df, client_mapping, how="left", on="client_sap_id")

    # Груз
    freight_mapping = pd.read_csv(
        freight_mapping_file, sep=";", decimal=",", encoding="windows-1251"
    )
    df = pd.merge(df, freight_mapping, how="left", on="freight_id")

    # Станции
    station_mapping = pd.read_csv(
        station_mapping_file,
        sep=";",
        decimal=",",
        encoding="windows-1251",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("sender_"),
        how="left",
        on="sender_station_id",
    )
    df = pd.merge(
        df,
        station_mapping.add_prefix("recipient_"),
        how="left",
        on="recipient_station_id",
    )

    return df


def evaluate(fact: pd.DataFrame, forecast: pd.DataFrame, public: bool = True) -> float:
    # = Параметры для расчета метрики =
    accuracy_granularity = [
        "period",
        "rps",
        "holding_name",
        "sender_department_name",
        "recipient_department_name",
    ]
    fact_value, forecast_value = "real_wagon_count", "forecast_wagon_count"
    if public:
        metric_weight = np.array([0.0, 1.0, 0.0, 0.0, 0.0])
    else:
        metric_weight = np.array([0.1, 0.6, 0.1, 0.1, 0.1])

    # = Собственно расчет метрик =
    # 1. Добавляем сущности верхних уровней гранулярности по справочникам
    fact = add_master_data_mappings(fact)
    forecast = add_master_data_mappings(forecast)

    # 2. Расчет KPI
    compare_data = pd.merge(
        fact.groupby(accuracy_granularity, as_index=False)[fact_value].sum(),
        forecast.groupby(accuracy_granularity, as_index=False)[forecast_value].sum(),
        how="outer",
        on=accuracy_granularity,
    ).fillna(0)
    # Против самых хитрых - нецелочисленный прогноз вагоноотправок не принимаем
    compare_data[fact_value] = np.around(compare_data[fact_value]).astype(int)
    compare_data[forecast_value] = np.around(compare_data[forecast_value]).astype(int)

    # 3. Рассчитаем метрики для каждого месяца в выборке
    compare_data["ABS_ERR"] = abs(
        compare_data[forecast_value] - compare_data[fact_value]
    )
    compare_data["MAX"] = abs(compare_data[[forecast_value, fact_value]].max(axis=1))
    summary = compare_data.groupby("period")[
        [forecast_value, fact_value, "ABS_ERR", "MAX"]
    ].sum()
    summary["Forecast Accuracy"] = 1 - summary["ABS_ERR"] / summary["MAX"]

    # 4. Взвесим метрики отдельных месяцев для получения одной цифры score
    score = (
        summary["Forecast Accuracy"].sort_index(ascending=True) * metric_weight
    ).sum()

    return score


def calc_score_public(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=True)


def calc_score_private(fact: pd.DataFrame, forecast: pd.DataFrame) -> float:
    return evaluate(fact, forecast, public=False)



In [20]:
def check_missings(df: pd.DataFrame) -> pd.DataFrame:
    """
    Функция для вычисления среднего и общего числа пропусков.

    Parameters
    ----------
    df: pandas.core.DataFrame
        Набор данных для вычисления статистики.

    Returns
    -------
    result: pandas.core.DataFrame
        Датафрейм со статистикой распределения пропусков.

    """
    na = df.isnull().sum()
    result = pd.DataFrame({
        "Total (всего пропусков)": na,
        "Percent (процент пропусков)": round(100*na/df.shape[0], 3),
        "Types (тип данных)": df.dtypes
    })
    result = result[result["Total (всего пропусков)"] != 0]
    print(f"Total NA-values (всего пропусков) = {na.sum()}")
    return result.T

In [21]:
check_missings(fact)

Total NA-values (всего пропусков) = 0


""
Total (всего пропусков)
Percent (процент пропусков)
Types (тип данных)


In [24]:
fact

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,71.0,1
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,210.0,3
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,67.0,1
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,67.0,1
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,66.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3559227,2023-03-01,1,5,2,1346,1482,31438,31482,0,69.0,1
3559228,2023-03-01,1,5,2,1346,1492,31438,36091,27275,70.0,1
3559229,2023-03-01,1,5,2,1346,1492,31438,35450,27275,70.0,1
3559230,2023-03-01,1,5,2,1346,1492,31438,31482,0,207.0,3


In [47]:
fact['podrod'] = fact['podrod'].apply(lambda x: 3 if x == 9 else x)
fact['podrod'].value_counts()

podrod
5    1847759
2     600137
0     510857
1     319948
4     280478
3         53
Name: count, dtype: int64

In [48]:
cat_features = ['period', 'rps', 'podrod', 'filial', 'client_sap_id', 'freight_id', 'sender_station_id', 'recipient_station_id', 'sender_organisation_id']

In [49]:
X = fact[cat_features]

In [50]:
y_weight = fact['real_weight']

In [51]:
y_count = fact['real_wagon_count']

In [70]:
params = {
        'verbose': 100,
        'random_seed': 42,
        'learning_rate': 0.1,
        'eval_metric': 'AUC',        
        'iterations':2000
        #'cat_features': cat_features, 
}

In [71]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(**params)

In [66]:
df = fact.drop('real_weight', axis=1)
df = df.drop('real_wagon_count', axis=1)

In [67]:
# Разделяем датафрейм на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df, fact["real_weight"], test_size=0.25)

In [72]:
model.fit(X_train, y_train)

0:	total: 189ms	remaining: 6m 18s
100:	total: 17.8s	remaining: 5m 34s
200:	total: 35.4s	remaining: 5m 17s
300:	total: 53.7s	remaining: 5m 3s
400:	total: 1m 12s	remaining: 4m 47s
500:	total: 1m 30s	remaining: 4m 30s
600:	total: 1m 49s	remaining: 4m 13s
700:	total: 2m 7s	remaining: 3m 56s
800:	total: 2m 26s	remaining: 3m 39s
900:	total: 2m 45s	remaining: 3m 22s
1000:	total: 3m 4s	remaining: 3m 4s
1100:	total: 3m 24s	remaining: 2m 46s
1200:	total: 3m 42s	remaining: 2m 28s
1300:	total: 4m	remaining: 2m 9s
1400:	total: 4m 19s	remaining: 1m 50s
1500:	total: 4m 37s	remaining: 1m 32s
1600:	total: 4m 56s	remaining: 1m 13s
1700:	total: 5m 14s	remaining: 55.2s
1800:	total: 5m 32s	remaining: 36.7s
1900:	total: 5m 50s	remaining: 18.3s
1999:	total: 6m 8s	remaining: 0us


In [73]:
score = model.score(X_test, y_test)
score

0.5957982130693116

In [76]:
# Разделяем датафрейм на обучающую и тестовую выборки
X_train2, X_test2, y_train2, y_test2 = train_test_split(fact, fact["real_wagon_count"], test_size=0.25)

In [77]:
model2 = CatBoostRegressor(**params)

In [78]:
model2.fit(X_train2, y_train2)

0:	total: 181ms	remaining: 6m 1s
100:	total: 18.3s	remaining: 5m 43s
200:	total: 34.8s	remaining: 5m 11s
300:	total: 51.6s	remaining: 4m 51s
400:	total: 1m 9s	remaining: 4m 38s
500:	total: 1m 29s	remaining: 4m 27s
600:	total: 1m 49s	remaining: 4m 14s
700:	total: 2m 8s	remaining: 3m 57s
800:	total: 2m 25s	remaining: 3m 37s
900:	total: 2m 42s	remaining: 3m 17s
1000:	total: 2m 59s	remaining: 2m 59s
1100:	total: 3m 17s	remaining: 2m 40s
1200:	total: 3m 33s	remaining: 2m 22s
1300:	total: 3m 51s	remaining: 2m 4s
1400:	total: 4m 8s	remaining: 1m 46s
1500:	total: 4m 26s	remaining: 1m 28s
1600:	total: 4m 44s	remaining: 1m 10s
1700:	total: 5m 1s	remaining: 53.1s
1800:	total: 5m 18s	remaining: 35.2s
1900:	total: 5m 35s	remaining: 17.5s
1999:	total: 5m 55s	remaining: 0us


In [79]:
score2 = model2.score(X_test2, y_test2)
score2

0.9886961496251983